In [1]:
import pandas as pd
import json
import numpy as np

In [90]:
full_data = pd.read_csv('dataset_building/8_11_14_18.csv')

In [108]:
prev_id=[]
beg_id=[]

for i, cong in enumerate([114,116, 118]):
    prev_idx = int(gt_112_data[gt_112_data.congress == cong-1].iloc[-1].name)
    # prev_id.append(prev_idx)
    
    # beg_idx = int(fourteen_18_data[fourteen_18_data.congress == cong].iloc[0].name)
    # beg_id.append(beg_idx)
    gt_112_data = pd.concat([gt_112_data[:prev_idx+1], fourteen_18_data[fourteen_18_data.congress==cong], gt_112_data[prev_idx+1:]]).reset_index(drop=True)
    # next_idx = int(gt_112_data[gt_112_data.congress == cong+1].iloc[0].name) + 1

In [147]:
last_ind = []
for i in [108, 109, 110, 111, 114, 116, 118]: # range(108, 119):
    last_idx = data[data.congress==i].iloc[-1].name
    last_ind.append(last_idx)
last_ind

[59, 440, 965, 1096, 1682, 2050, 2170]

In [17]:
with open('dataset_building/last_inds.json', 'r') as f:
    last_ind = json.load(f)['last_ind']
last_ind

[59, 440, 965, 1482, 1968, 2528, 3114, 3536, 3904, 4318, 4438]

In [16]:
with open('memory/qwen_run_8_11/detection.json', 'r+') as f:
    eight_el = json.load(f)
# with open('memory/yi_run_gt112/detection.json', 'r+') as f:
#     gt_112 = json.load(f)
with open('memory/qwen_run_14_16_18/detection.json', 'r+') as f:
    fourteen_118 = json.load(f)
with open('memory/qwen_run_gt112/detection.json', 'r+') as f:
    gt_112 = json.load(f)

to get results from incomplete 8_11 run

In [142]:
eight_11_incomp_14_18 = pd.concat([eight_11.iloc[:eight_el_main.keys().__len__()], fourteen_18]).reset_index(drop=True)
# eight_11_incomp_14_18.to_csv('dataset_building/eight_11_incomp_14_18.csv', index=False)

---

for those where i missed updating count while resuming

In [269]:
with open('memory/mixtral_cot_run_14_16_18/main.json', 'r+') as f:
    fourteen_118_main = json.load(f)

In [270]:
detected = []
for i in range(1074):
    keys = list(fourteen_118_main[str(i)].keys())
    keys.remove('benefactor')
    
    assert keys.__len__()<4
    
    if keys.__len__() == 1:
        detected.append(0)
    elif keys.__len__() == 2:
        detected.append(1)
    elif keys.__len__() == 3:
        if fourteen_118_main[str(i)]['2']['detected']:
            detected.append(3)
        else:
            detected.append(2)

---

In [18]:
full_detection = np.concatenate([np.array(eight_el), np.array(gt_112)[:gaps['gt_112_gap_prev'][0]], np.array(fourteen_118)[:gaps['14_18_gaps_begin'][1]], np.array(gt_112)[gaps['gt_112_gap_prev'][0]:gaps['gt_112_gap_prev'][1]], 
                np.array(fourteen_118)[gaps['14_18_gaps_begin'][1]:gaps['14_18_gaps_begin'][2]], np.array(gt_112)[gaps['gt_112_gap_prev'][1]:], np.array(fourteen_118)[gaps['14_18_gaps_begin'][2]:]
              ])#.shape

In [6]:
# with open('detection_results/qwen_detection.json', 'w') as f:
#     json.dump(full_detection.tolist(), f, indent=4)
with open('memory/robustness_trial/qwen14_run_14_16_18_200_rob_ablate_comp/detection.json', 'r') as f:
    full_detection = json.load(f)

dt = np.array(full_detection)

In [7]:
dt[:, :, 0].mean()

0.65

#### top-1

In [ ]:
import json
import numpy as np

detection_top1_all_models = {}
# for model in ['mistral7instructV2', 'qwen7', 'qwen14', 'yi34Chat', 'mixtral', 'qwen']:
for model in ['qwen14']:
    
    # with open(f'detection_results/{model}_detection.json', 'r') as f:
    with open(f'memory/constant_critic/smaller_14/qwen/poll_pairs_three_llms/poll_pairs/detection_14_500.json', 'r') as f:
        full_detection = json.load(f)

    dt = np.array(full_detection)

    for i in range(dt[:, :, 0].shape[0]):
        for j in range(dt[i, :, 0].shape[0]-2, -1, -1):
            if not dt[i, :, 0][j]:
                dt[i, :, 0][j:] = 0

    dt_top1 = dt[:, :, 0].copy()
    
    detection_per_trial = []
    for i in range(3):
        detection_per_trial.append(dt_top1[:, i].mean())
    # if model == 'qwen':
    #     detection_per_trial.append(dt_top1[:, 3].mean())
    detection_top1_all_models[model] = detection_per_trial

detection_top1_all_models

# with open('detection_results/qwen14_poll_pair_three_llm.json', 'w') as f:
#     json.dump(detection_top1_all_models, f, indent=4)

#### top-2

In [30]:
import json
with open(f'memory/qwen14_run_14_16_18/detection.json', 'r') as f:
    full_detection = json.load(f)

In [31]:
import numpy as np

det_results = []
for i in range(3): 
    det_results.append(np.array(full_detection)[:, i].sum(-1).mean())
    
det_results

[0.7970204841713222, 0.6340782122905028, 0.5726256983240223]

In [9]:
import json

with open('memory/constant_critic/larger/qwen14_run_14_16_18/top2_results.json', 'w') as f:
    json.dump(det_results, f, indent=4)

In [18]:
results = {}
gt = []
percents = []
for i in range(3):
    gt.append(np.count_nonzero(np.int32(dt.sum(1).sum(-1) > i)))#, np.count_nonzero(np.int32(dt_cot.sum(1).sum(-1) > 2))
    percents.append(gt[i]/dt.shape[0])
    
# with open('dataset_building/last_inds.json', 'r') as f:
#     last_ind = json.load(f)['last_ind']


cong_gt = []
cong_percents = []
lst = [-1] + last_ind
ranges = [[lst[i]+1, lst[i+1]+1] for i in range(len(lst)-1)]
for r in ranges:
    cong_gt += [[]]
    cong_percents += [[]]
    cong_dt = dt[r[0]:r[1]]
    for i in range(3):
        cong_gt[-1].append(np.count_nonzero(np.int32(cong_dt.sum(1).sum(-1) > i)))
        # cong_gt[-1].append(np.count_nonzero(np.int32(cong_dt > i)))
        
        cong_percents[-1].append(cong_gt[-1][i]/cong_dt.shape[0])

results['gt'] = gt
results['percents'] = percents
results['cong_gt'] = cong_gt
results['cong_percents'] = cong_percents

In [150]:
states = data.state.unique()
state_ind = {}
for s in states:
    state_ind[s] = data[data.state==s].index
# state_ind

In [151]:
state_gt = {}
state_percents = {}

for k,v in state_ind.items():
    state_gt[k] = []
    state_percents[k] = []
    for i in range(3):
        state_gt[k] += [np.count_nonzero(np.int32(dt[v].sum(1).sum(-1) > i))]
        # state_gt[k] += [np.count_nonzero(np.int32(dt[v] > i))]
        
        state_percents[k] += [state_gt[k][i]/dt[v].shape[0]]
        
results['state_gt'] = state_gt
results['state_percents'] = state_percents

In [152]:
policy = [a for a in data.policyArea.unique() if a is not np.nan]
policy_ind = {}
for p in policy:
    policy_ind[p] = data[data.policyArea==p].index

policy_gt = {}
policy_percents = {}

for k,v in policy_ind.items():
    policy_gt[k] = []
    policy_percents[k] = []
    for i in range(3):
        policy_gt[k] += [np.count_nonzero(np.int32(dt[v].sum(1).sum(-1) > i))]
        # policy_gt[k] += [np.count_nonzero(np.int32(dt[v]) > i)]
        
        policy_percents[k] += [policy_gt[k][i]/dt[v].shape[0]]
        
results['policy_gt'] = policy_gt
results['policy_percents'] = policy_percents

In [154]:
with open('detection_results/yiChat_8_11_incomp_14_16_18_results.json', 'w') as f:
    json.dump(results, f, indent=4)

gu, pr, as → territories, not states\
dc → district (washinton dc)

##### Top 1 states

In [84]:
import pandas as pd
with open(f'memory/qwen_run_8_11/detection.json', 'r') as f:
    dt = np.array(json.load(f))
    
data = pd.read_csv('dataset_building/8_11/selected_8_11_policya_state.csv')

state_detections = {}

for i in range(dt[:, :, 0].shape[0]):
        for j in range(dt[i, :, 0].shape[0]-2, -1, -1):
            if not dt[i, :, 0][j]:
                dt[i, :, 0][j:] = 0
                
dt_top1 = dt[:, :3, 0].copy()

for state, det in zip(data.state, dt_top1):
    state_detections[state] = state_detections.get(state, []) + [det.tolist()]

In [92]:
import json
import pandas as pd
import numpy as np

datamap = {'8_11':'dataset_building/8_11/selected_8_11_policya_state.csv',
           'gt112': 'dataset_building/gt112_except14_16_18/gt112_policya_state.csv',
           '14_16_18': 'dataset_building/14_16_18/14_16_18_policya_state.csv'}

state_detections = {}
state_detection_percents = {}

for split in ['8_11', 'gt112', '14_16_18']:
    with open(f'memory/qwen_run_{split}/detection.json', 'r') as f:
        dt = np.array(json.load(f))

    data = pd.read_csv(datamap[split])
    
    for i in range(dt[:, :, 0].shape[0]):
        for j in range(dt[i, :, 0].shape[0]-2, -1, -1):
            if not dt[i, :, 0][j]:
                dt[i, :, 0][j:] = 0
                
    dt_top1 = dt[:, :3, 0].copy()


    for state, det in zip(data.state, dt_top1):
        state_detections[state] = state_detections.get(state, []) + [det.tolist()]
        
for state in state_detections:
    for trial in range(3):
        state_detection_percents[state] = state_detection_percents.get(state, []) + [np.array(state_detections[state])[:, trial].mean()]

#### For Plotting results

In [159]:
with open('detection_results/mixtral_run_results.json', 'r') as f:
    mixtral_results = json.load(f)
with open('detection_results/qwen_results.json', 'r') as f:
    qwen_results = json.load(f)
with open('detection_results/yiChat_8_11_incomp_14_16_18_results.json', 'r') as f:
    yi_results = json.load(f)

##### policy areas

In [80]:
pa_list = []
for p, det in results['policy_gt'].items():
    if det[0]>20:
        print(p, det)
        pa_list.append(p)

International affairs [289, 213, 170]
Government operations and politics [223, 177, 161]
Health [347, 230, 193]
Armed forces and national security [129, 91, 79]
Energy [73, 55, 47]
Agriculture and food [116, 86, 73]
Immigration [183, 138, 109]
Law [42, 32, 27]
Transportation and public works [167, 121, 96]
Crime and law enforcement [188, 137, 115]
Public lands and natural resources [212, 152, 133]
Foreign trade and international finance [33, 20, 19]
Commerce [67, 53, 46]
Water resources development [48, 32, 28]
Environmental protection [136, 91, 79]
Emergency management [121, 82, 66]
Native Americans [68, 44, 40]
Congress [43, 26, 25]
Labor and employment [65, 47, 41]
Economics and public finance [75, 59, 44]
Science, technology, communications [77, 52, 46]
Finance and financial sector [47, 30, 28]
Education [43, 32, 26]
Animals [24, 20, 16]
Social welfare [28, 18, 18]
Civil rights and liberties, minority issues [47, 40, 32]


In [157]:
policy_per = {}
for p, per in yi_results['policy_percents'].items():
    # if p in pa_list:
    print(p, per, yi_results['policy_gt'][p])
    policy_per[p] = per + yi_results['policy_gt'][p]

International affairs [0.8393782383419689, 0.7823834196891192, 0.7202072538860104] [162, 151, 139]
Government operations and politics [0.7361111111111112, 0.6597222222222222, 0.5972222222222222] [106, 95, 86]
Health [0.7565543071161048, 0.6629213483146067, 0.6179775280898876] [202, 177, 165]
Armed forces and national security [0.8133333333333334, 0.7333333333333333, 0.72] [61, 55, 54]
Energy [0.6039603960396039, 0.5445544554455446, 0.49504950495049505] [61, 55, 50]
Agriculture and food [0.6136363636363636, 0.5681818181818182, 0.5454545454545454] [54, 50, 48]
Immigration [0.8410596026490066, 0.7615894039735099, 0.7284768211920529] [127, 115, 110]
Law [0.7741935483870968, 0.7419354838709677, 0.7419354838709677] [24, 23, 23]
Transportation and public works [0.6601307189542484, 0.5751633986928104, 0.5490196078431373] [101, 88, 84]
Crime and law enforcement [0.7682119205298014, 0.6754966887417219, 0.6225165562913907] [116, 102, 94]
Public lands and natural resources [0.6260162601626016, 0.5

In [158]:
pd.DataFrame(policy_per).transpose().to_csv('detection_results/yi_policy.csv')

##### congress

In [165]:
cong_per = {}
for p, per in enumerate(qwen_results['cong_percents']):
    # if p in pa_list:
    print(p, per, qwen_results['cong_gt'][p])
    cong_per[p] = per + qwen_results['cong_gt'][p]

0 [0.9166666666666666, 0.8, 0.6] [55, 48, 36]
1 [0.9186351706036745, 0.7716535433070866, 0.6010498687664042] [350, 294, 229]
2 [0.9352380952380952, 0.8114285714285714, 0.6628571428571428] [491, 426, 348]
3 [0.9284332688588007, 0.7911025145067698, 0.6170212765957447] [480, 409, 319]
4 [0.9053497942386831, 0.7716049382716049, 0.6213991769547325] [440, 375, 302]
5 [0.9053571428571429, 0.7696428571428572, 0.6267857142857143] [507, 431, 351]
6 [0.9215017064846417, 0.7815699658703071, 0.643344709897611] [540, 458, 377]
7 [0.9289099526066351, 0.7962085308056872, 0.6635071090047393] [392, 336, 280]
8 [0.9347826086956522, 0.8179347826086957, 0.6902173913043478] [344, 301, 254]
9 [0.9299516908212561, 0.8043478260869565, 0.6763285024154589] [385, 333, 280]
10 [0.875, 0.6833333333333333, 0.55] [105, 82, 66]



#### States

In [175]:
state_per = {}
for p, per in mixtral_results['state_percents'].items():
    # if p in pa_list:
    print(p, per, mixtral_results['state_gt'][p])
    state_per[p] = per + mixtral_results['state_gt'][p]

NJ [0.7437185929648241, 0.5226130653266332, 0.4371859296482412] [148, 104, 87]
CA [0.6891891891891891, 0.481981981981982, 0.40765765765765766] [306, 214, 181]
PA [0.640625, 0.4921875, 0.46875] [82, 63, 60]
FL [0.6387096774193548, 0.5032258064516129, 0.4] [99, 78, 62]
IL [0.6949152542372882, 0.5310734463276836, 0.4576271186440678] [123, 94, 81]
MA [0.7169811320754716, 0.5660377358490566, 0.5] [76, 60, 53]
TX [0.6266666666666667, 0.4622222222222222, 0.3688888888888889] [141, 104, 83]
NY [0.6951219512195121, 0.45934959349593496, 0.36585365853658536] [171, 113, 90]
MO [0.6086956521739131, 0.4782608695652174, 0.42028985507246375] [42, 33, 29]
SC [0.7777777777777778, 0.5111111111111111, 0.4444444444444444] [35, 23, 20]
MD [0.6363636363636364, 0.45454545454545453, 0.375] [56, 40, 33]
WI [0.8225806451612904, 0.5806451612903226, 0.4838709677419355] [51, 36, 30]
NH [0.65, 0.5, 0.475] [26, 20, 19]
VA [0.635036496350365, 0.49635036496350365, 0.41605839416058393] [87, 68, 57]
MS [0.6774193548387096

In [176]:
# pd.DataFrame(state_per).transpose().to_csv('detection_results/mixtral_state.csv')

#### Bootstrap Confidence Intervals

##### prerequisites

In [5]:
import sklearn as sk
import json
import numpy as np

import numpy as np
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a[0])
    # m, se = np.mean(a, axis=-1), scipy.stats.sem(a, axis=-1)
    m, se = np.mean(a, axis=-1), scipy.stats.tstd(a, axis=-1)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return [[list(m), list(m-h), list(m+h)], list(se)]
    # return [[m, m-h, m+h], se]

In [7]:
with open(f'memory/ablation/qwen14_no_deception/detection.json', 'r') as f:
        detection = json.load(f)
boot = np.array(sk.utils.resample(detection, replace=True, n_samples=detection.__len__(), random_state=42))

##### top-2

In [19]:
from tqdm import tqdm
import json

detection_bootstrap_dict = {}
confidence_intervals = {}

# def bootstrap(model:str):
for model in ['qwen', 'yiChat_8_11_14_18', 'mixtral', 'qwen14', 'qwen7', 'mistral7instructV2']:
# for model in ['mistral7instructV2', 'qwen7', 'qwen14', 'yi', 'mixtral']:
    # with open(f'detection_results/{model}_detection.json', 'r') as f:
    with open(f'memory/{model}_run_{split}/detection.json', 'r') as f:
        detection = json.load(f)
    
    detection_mean = [[],[],[]]
    if model == 'qwen':
        detection_mean.append([])
    for i in tqdm(range(10000), desc=model):
        boot = sk.utils.resample(detection, replace=True, n_samples=detection.__len__(), random_state=i)
        
        detection_mean[0].append(np.array(boot)[:, 0].sum(-1).mean())
        detection_mean[1].append(np.array(boot)[:, 1].sum(-1).mean())
        detection_mean[2].append(np.array(boot)[:, 2].sum(-1).mean())
        if model =='qwen':
            detection_mean[3].append(np.array(boot)[:, 3].sum(-1).mean())
        
    
    detection_bootstrap_dict[model] = detection_mean
    confidence_intervals[model] = [np.array(detection)[:, i].sum(-1).mean() for i in range(3)] + [mean_confidence_interval(detection_bootstrap_dict[model])]
    
    # with open('detection_results/bootstrap/constant_critic/top2_bootstrap_all_models_bs10k.json', 'w') as f:
    #     json.dump(detection_bootstrap_dict, f)
    # with open('detection_results/bootstrap/constant_critic/top2_confidence_intervals_bs10k.json', 'w') as f:
    #     json.dump(confidence_intervals, f, indent=4)

mixtral: 100%|██████████| 10000/10000 [00:28<00:00, 356.83it/s]


##### top-1

In [20]:
from tqdm import tqdm
import json

top1_detection_bootstrap_dict = {}
top1_confidence_intervals = {}

# for model in ['qwen', 'yiChat_8_11_14_18', 'mixtral', 'qwen14', 'qwen7', 'mistral7instructV2']:
# for model in ['crit_comp_reverse', 'crit_desc_reverse', 'crit_remove_desc', 'rob_ablate_comp', 'rob_bus_bil']:
for model in ['qwen14_ablate']:

    # if model=='qwen14_ablate':
    #     with open(f'memory/ablation/qwen14_ablate_plan/detection_ablate_plan.json', 'r') as f:
    #         detection = json.load(f)
    # else:
    with open(f'memory/constant_critic/smaller_14/qwen/poll_pairs_three_llms/poll_pairs/detection_14_500.json', 'r') as f:
        detection = json.load(f)
        
    detection_mean = [[], [], []]
    # if model == 'qwen':
    #     detection_mean.append([])
    for i in tqdm(range(5000), desc=model):
        boot = np.array(sk.utils.resample(detection, replace=True, n_samples=detection.__len__(), random_state=i))
        
        for i in range(boot[:, :, 0].shape[0]):
            for j in range(boot[i, :, 0].shape[0]-2, -1, -1):
                if not boot[i, :, 0][j]:
                    boot[i, :, 0][j:] = 0

        boot_top1 = boot[:, :, 0].copy()
        
        for i in range(3):
            detection_mean[i].append(boot_top1[:, i].mean())
        # if model == 'qwen':
        #     detection_mean[3].append(boot_top1[:, 3].mean())
    
    detection = np.array(detection)
    for i in range(detection[:, :, 0].shape[0]):
        for j in range(detection[i, :, 0].shape[0]-2, -1, -1):
            if not detection[i, :, 0][j]:
                detection[i, :, 0][j:] = 0
                
    detection_top1 = detection[:, :, 0].copy()
    
    # detection_per_trial = []
    top1_detection_bootstrap_dict[model] = detection_mean
    top1_confidence_intervals[model] = [detection_top1[:, i].mean() for i in range(3)] + [mean_confidence_interval(top1_detection_bootstrap_dict[model])]
    
    # with open('detection_results/bootstrap/constant_critic/top1_bootstrap_all_models_bs10k.json', 'w') as f:
    #     json.dump(top1_detection_bootstrap_dict, f)
    with open('detection_results/bootstrap/qwen14_no_deception.json', 'w') as f:
        json.dump(top1_confidence_intervals, f, indent=4)

qwen14_ablate: 100%|██████████| 5000/5000 [00:06<00:00, 720.57it/s]


In [21]:
top1_confidence_intervals

{'qwen14_ablate': [0.678,
  0.456,
  0.294,
  [[[0.6782556, 0.45628159999999995, 0.2941708],
    [0.6371158455654001, 0.4129397793713246, 0.25436164721873494],
    [0.7193953544345998, 0.4996234206286753, 0.3339799527812651]],
   [0.020984974207000892, 0.02210822598426283, 0.020306247710968655]]]}

In [ ]:
# np.mean(detection_bootstrap_dict['qwen'],axis=-1) - [0.9, 0.7, 0.6]
# confidence_intervals = {}

# for model in ['qwen14', 'qwen', '14_qwen']:
    
#     confidence_intervals[model] = mean_confidence_interval(detection_bootstrap_dict[model])

with open('detection_results/bootstrap/constant_critic/smaller_7_qwen14_qwen_14_qwen.json', 'w') as f:
    json.dump(top1_confidence_intervals, f, indent=4)
confidence_intervals

NameError: name 'confidence_intervals' is not defined

##### States

Qwen 72B

In [114]:
import json
import pandas as pd
import numpy as np
import sklearn as sk

datamap = {'8_11':'dataset_building/8_11/selected_8_11_policya_state.csv',
           'gt112': 'dataset_building/gt112_except14_16_18/gt112_policya_state.csv',
           '14_16_18': 'dataset_building/14_16_18/14_16_18_policya_state.csv'}

state_detections = {}
state_detection_percents = {}

state_ci = {}

for split in ['14_16_18']:
    with open(f'memory/qwen_run_{split}/detection.json', 'r') as f:
        dt = np.array(json.load(f))

    data = pd.read_csv(datamap[split])
    
    for i in range(dt[:, :, 0].shape[0]):
        for j in range(dt[i, :, 0].shape[0]-2, -1, -1):
            if not dt[i, :, 0][j]:
                dt[i, :, 0][j:] = 0
                
    dt_top1 = dt[:, :3, 0].copy()


    for state, det in zip(data.state, dt_top1):
        state_detections[state] = state_detections.get(state, []) + [det.tolist()]
        
for state in state_detections:
    
    
    for trial in range(3):
        state_detection_percents[state] = state_detection_percents.get(state, []) + [np.array(state_detections[state])[:, trial].mean()]
    
    
    detection_mean = [[], [], []]
    boots = []
    for i in tqdm(range(10000), desc=state):
        boot = np.array(sk.utils.resample(state_detections[state], replace=True, n_samples=state_detections[state].__len__(), random_state=i))
        # means.append(boot.mean())
        for trial in range(3):
            detection_mean[trial] += [boot[:, trial].mean()]
        
    state_ci[state] = [state_detection_percents[state]] + [mean_confidence_interval(detection_mean)]
        

NM: 100%|██████████| 10000/10000 [00:02<00:00, 3817.26it/s]


In [115]:
with open('detection_results/bootstrap/states/qwen_14_16_18_states_ci.json', 'w') as f:
    json.dump(state_ci, f, indent=4)

In [116]:
# for state in state_ci:
#     state_ci[state] = [i for i in state_ci[state][0]] + [i for i in state_ci[state][1][1]]

In [118]:
states_detection_sd = pd.DataFrame(state_ci).transpose()
states_detection_sd['deception'] = states_detection_sd[0] - states_detection_sd[2]

In [119]:
states_detection_sd.sort_values('deception').to_csv('states_analysis/states_ranking_14_16_18.csv')

#### Verifying constant critic results

In [22]:
length_upto_14 = 0
for i in range(108, 114):
    length_upto_14 += full_data[full_data.congress == i].__len__()
lengths=[]
for i in [114, 115, 116, 117, 118]:
    lengths += [full_data[full_data.congress == i].__len__()]
    
length_upto_14, lengths

(2529, [586, 422, 368, 414, 120])

In [23]:
# with open('memory/constant_critic/larger/qwen14/detection.json', 'r') as f:
#     detection = json.load(f)

In [70]:
# extracting index ranges for 14_16_18 records

ranges = [[length_upto_14, length_upto_14+lengths[0]], [length_upto_14+sum(lengths[:2]), length_upto_14+sum(lengths[:3])], [length_upto_14+sum(lengths[:4]), length_upto_14+sum(lengths[:5])]]

detection_14 = []
for r in ranges:
    detection_14 += detection[r[0]:r[1]]

In [71]:
np.array(detection_14)[:, 0].sum(-1).mean(), np.array(detection)[:, 0].sum(-1).mean()

(0.41899441340782123, 0.5676954268979499)

Experiments for constant critic with qwen14 show a major drop (above). The above are results for full dataset and 14-18 split.\\While simply doing this on the stronger critic script with qwen 72b q5_k_m give higher results for 14-18 splits (aprox 0.76)

In [100]:
import json
with open('memory/constant_critic/prev_yi_main.json', 'r') as f:
    main = json.load(f)

In [101]:
main.__len__()

4439

In [93]:
benefactors_in_main = [main[k]['benefactor'] for k in main]

In [92]:
full_data_company_names = list(full_data.company_name)

In [97]:
full_data_company_names.__len__()

2557

In [94]:
match_fail_idx = []
for idx, (x,y) in enumerate(zip(benefactors_in_main, full_data_company_names)):
    if x != y:
       match_fail_idx.append(idx)

In [98]:
match_fail_idx.__len__()

1071